In [ ]:
import torch
import  torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

#Setting Up DataLoaders



1.   ImageNet is a standard collection of images used to train neural networks. It contains more than 14 million images and 20,000 image categories.
2.   A dataset is a python class that allows us to get at the data we are supplying to the neural network.
3.  A data loader is what feeds data from the dataset into the network.




*   Built-n dataset of torchvision.datasets.ImageFolder to quickly set up some dataloaders of download cat and fish images

*   Check_image is a quick little functions that is passed to the is_valid_file parameter in the ImageFolder and will do a sanity check to make sure PIL can actually open the file.




In [ ]:
def check_image(path):
  try:
    im = Image.open(path)
    return True
  except:
    return False()



*   Resize to 64*64
*   Convert to tensor
*   Normalize using ImageNet mean & std





In [ ]:
image_transformation = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406],
                        std=[0.229,0.224,0.225])
])

Torchvision package includes a class called ImageFolder that does pretty much 

*  Torchvision package includes a class called ImageFolder that does pretty much everything , providing images are in a structure where each directory is a label.
*   It allows us to specify a list of transforms that will be applied to an image before it gets fed into the neural network.The Default transform is to take image data and turn it into a tensor.
*   Training set - Used in the training pass to update the model
*   Validation set- Used to Evaluate how the model is generalizing to the problem domain, rather than fitting to the training data. It is not used to update the model directly.
*   Test Set - A final dataset that provides a final evaluation of the model's performance after training its complete.





In [ ]:
train_data_path = "/content/drive/MyDrive/Datasets/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=image_transformation,is_valid_file = check_image)

In [ ]:
validation_data_path = "/content/drive/MyDrive/Datasets/val/"
validation_data = torchvision.datasets.ImageFolder(root=validation_data_path,transform=image_transformation,is_valid_file =  check_image)

In [ ]:
test_data_path = "/content/drive/MyDrive/Datasets/test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform = image_transformation,is_valid_file = check_image)


*   Batch Size tells how  many images will go through the network before we train and update it.
*   By default Pytorch's data loader are set to batch_size of 1.



In [ ]:
batch_size = 64

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_data,batch_size = batch_size)
validation_data_loader = torch.utils.data.DataLoader(validation_data,batch_size = batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data,batch_size = batch_size)



*   Activation functions is a way of inserting non-linearity into our system.



#First Model SimpleNet


1.   SimpleNet is a very simple combination of three linear layers and ReLu activations betwenn them.
2.   Class torch.nn.Linear(in_features,out_features,bias=True,device=None,dtype=None)

      *   Applies a Linear Transformation to the incoming data y = x*AT + b
      *   in_features - size of each input sample
      *   out_feature - size of each output sample
      *   bias - if set to False , the layer will not learn an additive bias.Default: True
      *   Shape - Input(*,Hin) where * means any number of  dimensions includng None and Hin = in_features.
      *   Output:(*,Hout) where all but the last dimensions are the same shape as the input and Hout = out_features
      *   Linear.weight - The learnable weights of the module of shape(out_features,in_features).The values are initialized from U(-sqrt(k),sqrt(k)),
      where k = 1/in_features.
      *  Linear.bias = the learnable bias of the shape (out_features). initialized from U(-sqrt(k),sqrt(k)) where k = 1/in_features.


3.   Init function calls the superclass constructor and the three fully connected layers.
4.   Forward method describes how data flows through the network in both training and making predictionns(inferences).















In [ ]:
class SimpleNet(nn.Module):

  def __init__(self):
    super(SimpleNet,self).__init__()
    self.fc1 = nn.Linear(12288,84)
    self.fc2 = nn.Linear(84,50)
    self.fc3 = nn.Linear(50,2)

  def forward(self,x):
    x = x.view(-1,12288)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [ ]:
simpleNetObect  = SimpleNet()

#Loss Functions


*   CrossEntropyLoss is the built-in loss function recommeneded for Multi-Class Categorization.
*   It also incorporates softmax() as part of its operation.





#Create an Optimizer


*   To perform the updates on the neural network , we use an optimizer.
*   Issues - Trapped in local minima.
*   Key Improvements that Adam makes is that it uses a learning rate per parameter and adapts that learning rate depending on the rate of change of those parameters.
*   It keeps an exponentially decaying list of gradients and the square of those gradients and uses those to scale the global learning rate that Adam is working with.
*   List item







In [ ]:
optimizer = optim.Adam(simpleNetObect.parameters(),lr=0.001)

#Copy the model to CPU


*   Copy the model to the GPU if available
*   PyTorch by default does CPU based calculations.
*   To take advantage of the GPU, we need to move our input tensors and the model iself to the GPU by explicitly using the to() method.





In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

simpleNetObect.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

#Training


*   Trains the model
*   Copying batches to the GPU if required
*   Calculating Loss
*   Optimizing the network and perform validation at each epoch.


*   To compute the gradients,we call the backward() method on the model.
*   The optimizer.step() method uses those gradients afterward to perform the adjustments of the weight.
*   It turns out that the calculated gradients accumulate by default meaning, if we didn't zero the gradients at the end of the batch's iteration  the next batch would have to deal with this batch's gradients as well as its own and the batch after that would have to cope wiwth the previous two, and so on.
*   Use zero_grad()to make sure they are reset to zero after we are done with our loop.










In [ ]:
def train(model,optimizer,loss_fn,train_loader,val_loader,epochs = 20,device="cpu"):
  for epoch in range(1,epochs+1):
    training_loss = 0.0
    validation_loss = 0.0
    model.train()

    for batch in train_loader:
      optimizer.zero_grad()
      inputs,targets = batch
      inputs = inputs.to(device)
      targets = targets.to(device)
      output = model(inputs)
      loss = loss_fn(output,targets)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item() * inputs.size(0)
    training_loss /= len(train_loader.dataset)

    model.eval()
    num_correct = 0
    num_examples = 0

    for batch in val_loader:
      inputs , targets = batch
      inputs = inputs.to(device)
      output = model(inputs)
      targets = targets.to(device)
      loss = loss_fn(output,targets)
      validation_loss += loss.data.item() * inputs.size(0)
      correct = torch.eq(torch.max(F.softmax(output,dim=1),dim=1)[1],targets)
      num_correct += torch.sum(correct).item()
      num_examples += correct.shape[0]
    validation_loss /= len(validation_data_loader.dataset)

    print('Epoch:{},Training Loss :{:.2f},Validation Loss: {:.2f},accuracy = {:.2f}'.format(epoch,training_loss,validation_loss,num_correct/num_examples))


In [ ]:
train(simpleNetObect,optimizer,torch.nn.CrossEntropyLoss(),train_data_loader,validation_data_loader,epochs = 20 , device = device)

Epoch:1,Training Loss :2.35,Validation Loss: 6.26,accuracy = 0.29
Epoch:2,Training Loss :3.21,Validation Loss: 1.69,accuracy = 0.69
Epoch:3,Training Loss :1.22,Validation Loss: 0.94,accuracy = 0.50
Epoch:4,Training Loss :0.69,Validation Loss: 0.63,accuracy = 0.75
Epoch:5,Training Loss :0.42,Validation Loss: 0.71,accuracy = 0.68
Epoch:6,Training Loss :0.40,Validation Loss: 0.63,accuracy = 0.78
Epoch:7,Training Loss :0.29,Validation Loss: 0.68,accuracy = 0.75
Epoch:8,Training Loss :0.29,Validation Loss: 0.64,accuracy = 0.75
Epoch:9,Training Loss :0.23,Validation Loss: 0.68,accuracy = 0.74
Epoch:10,Training Loss :0.22,Validation Loss: 0.69,accuracy = 0.73
Epoch:11,Training Loss :0.20,Validation Loss: 0.68,accuracy = 0.73
Epoch:12,Training Loss :0.16,Validation Loss: 0.72,accuracy = 0.73
Epoch:13,Training Loss :0.16,Validation Loss: 0.69,accuracy = 0.74
Epoch:14,Training Loss :0.13,Validation Loss: 0.73,accuracy = 0.72
Epoch:15,Training Loss :0.12,Validation Loss: 0.73,accuracy = 0.73
Epoc

#Making Prediction


*   Unsqueeze adds a new dimension at the front of our tensor
*   Argmax returns the index of the highest value of the tensor.



In [ ]:
labels = ['cat','fish']
image = Image.open("/content/drive/MyDrive/Datasets/test/cat/261385269_5f0e6578b2.jpg")
image = image_transformation(image).to(device)
image = torch.unsqueeze(image,0)

simpleNetObect.eval()
prediction = F.softmax(simpleNetObect(image),dim=1)
prediction = prediction.argmax()
print(labels[prediction])

cat


#Saving Models

*   Save the model using save or just the parameters using state_dic.
*   save function store the current parameters and model structure.
*   Load function stores both the parameters and the structure of the model to a file.This might be a problem if you change the structure of the model at a later point.
*   It is more common to save a model's state_dict instead.This is a standard python dict that contains the map of each layer's parameters in the model.





In [ ]:
torch.save(simpleNetObect, "/content/drive/MyDrive/Colab Notebooks/Pytorch/temp/Model.docx")
simpleNetObect = torch.load("/content/drive/MyDrive/Colab Notebooks/Pytorch/temp/Model.docx")

In [ ]:
torch.save(simpleNetObect.state_dict(),"/content/drive/MyDrive/Colab Notebooks/Pytorch/temp/Model.docx")
simpleNetObect = SimpleNet()
simpleNetObect_state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/Pytorch/temp/Model.docx")
simpleNetObect.load_state_dict(simpleNetObect_state_dict)

<All keys matched successfully>